<a href="https://colab.research.google.com/github/T1D3S-HUB/ALURA-Sistema-de-An-lise-e-Desenvolvimento-de-Aprimoramentos-Kaizen/blob/main/C%C3%B3pia_de_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Projeto_de_kaizen_interativo_din%C3%A2mico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_FOR_AGENTS = "gemini-2.0-flash"

In [ ]:
!pip install -q google-adk

In [13]:
# Passo 0: Instalação e Configuração das Bibliotecas
# Certifique-se de executar esta célula primeiro no Colab, se necessário.
# !pip install -q google-generativeai google-ai-generativelanguage google-adk

import os
import textwrap
from datetime import datetime, date # Adicionado date para compatibilidade com o exemplo do usuário
from IPython.display import display, Markdown, HTML

# Tente importar userdata, se falhar (rodando localmente, por exemplo), defina a API Key manualmente
try:
    from google.colab import userdata
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    print("API Key carregada a partir do Colab userdata.")
except ImportError:
    # Se não estiver no Colab, você pode definir a variável de ambiente GOOGLE_API_KEY de outra forma
    # ou colar a chave diretamente aqui (menos seguro)
    # Exemplo: os.environ["GOOGLE_API_KEY"] = "SUA_API_KEY_AQUI"
    if "GOOGLE_API_KEY" not in os.environ:
        print("AVISO: GOOGLE_API_KEY não encontrada. Por favor, configure-a no seu ambiente.")
    else:
        print("API Key carregada a partir das variáveis de ambiente.")

# Configura o cliente da SDK do Gemini

    from google import genai
    from google.adk.agents import Agent
    from google.adk.runners import Runner
    from google.adk.sessions import InMemorySessionService
    from google.adk.tools import google_search
    from google.ai import generativelanguage as glm # Para criar conteúdos (Content e Part)
    # Ajuste para usar types de google.generativeai se glm não for o caminho correto para Content/Part
    # Com a SDK mais recente, é mais provável que seja:
    from google.generativeai.types import Content, Part

    genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))
    # MODEL_FOR_AGENTS = "gemini-1.5-flash-latest" # Modelo sugerido para os agentes
    MODEL_FOR_AGENTS = "gemini-2.0-flash" # Usando o modelo que estava no exemplo de agents do usuário
    print(f"Usando o modelo {MODEL_FOR_AGENTS} para os agentes.")

except ImportError as e:
    print(f"Erro ao importar bibliotecas necessárias: {e}")
    print("Certifique-se de ter instalado 'google-generativeai' e 'google-adk'.")
    print("Você pode precisar executar: !pip install -q google-generativeai google-adk")
except Exception as e:
    print(f"Erro durante a configuração do GenAI: {e}")


# Passo 1: Simulação de um banco de dados em memória para ideias processadas
banco_de_ideias_processadas = []

# Passo 2: Funções Auxiliares do Sistema ALURA (coleta de dados do usuário)

def mostrar_logo():
    """Exibe o logotipo do ALURA."""
    print("*"*42)
    print("* ALURA *")
    print("* (Sistema de Análise e *")
    print("* Desenvolvimento de Aprimoramentos) *")
    print("*"*42)
    print("\n")

def iniciar_chat_coleta_funcionario():
    """Inicia a conversa e coleta dados iniciais do funcionário."""
    mostrar_logo()
    print("Bem-vindo ao ALURA! O seu Sistema Inteligente para Ideias Kaizen.")
    print("Vamos registrar sua ideia para torná-la ainda melhor!\n")

    while True:
        nome = input("Para começarmos, por favor, informe seu nome completo: ").strip()
        if nome:
            break
        else:
            print("Nome inválido. Por favor, digite seu nome.")

    while True:
        matricula = input(f"Olá, {nome}! Agora, por favor, informe sua matrícula (apenas números): ").strip()
        if matricula.isdigit():
            break
        else:
            print("Matrícula inválida. Por favor, digite apenas números.")

    while True:
        setor_funcionario = input(f"Obrigado, {nome}! Em qual setor você trabalha? ").strip()
        if setor_funcionario:
            break
        else:
            print("Setor inválido. Por favor, digite o setor.")

    while True:
        local_problema = input(f"Em qual local específico (ou subprocesso) dentro do setor '{setor_funcionario}' (ou de outro setor) você identificou a oportunidade de melhoria? ").strip()
        if local_problema:
            break
        else:
            print("Local/subprocesso inválido. Por favor, especifique onde a melhoria se aplica.")

    return {"nome": nome, "matricula": matricula, "setor_funcionario": setor_funcionario, "local_problema": local_problema}

def coletar_ideia_bruta():
    """Coleta a descrição do problema e da solução proposta pelo funcionário."""
    print("\n--- Detalhes da Sua Ideia Kaizen ---")
    while True:
        problema = input("Descreva a ideia Kaizen que você tem. Qual é o problema ou oportunidade de melhoria que você identificou?\n(Seja o mais detalhado possível)\n").strip()
        if problema:
            break
        else:
            print("A descrição do problema não pode estar vazia.")

    while True:
        solucao = input("Interessante! E qual seria a sua proposta de solução para este problema/oportunidade?\n(Descreva sua sugestão de como resolver ou melhorar)\n").strip()
        if solucao:
            break
        else:
            print("A descrição da solução não pode estar vazia.")
    return {"problema": problema, "solucao": solucao}

# Passo 3: Funções Auxiliares para Agentes (call_agent e formatação)

def call_agent_sync(agent: Agent, message_text: str) -> str:
    """
    Função síncrona simplificada para chamar um agente e obter a resposta final.
    Nota: A função original do usuário era assíncrona na iteração,
    esta é uma adaptação para um fluxo mais direto, assumindo que a ADK permite.
    Se a ADK exigir iteração assíncrona, a função original do usuário é mais adequada.
    Esta versão é uma simplificação conceitual.
    """
    # Para a google-adk, o Runner.run() é um gerador, então precisamos iterar.
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user_ALURA", session_id="session_ALURA")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)

    # google.generativeai.types.Content e Part
    content = Content(role="user", parts=[Part(text=message_text)])

    final_response_text = ""
    print(f"\n🤖 Chamando Agente: {agent.name}...")
    try:
        # O método run é um gerador
        for event in runner.run(user_id="user_ALURA", session_id="session_ALURA", new_message=content):
            if event.is_final_response():
                for part in event.content.parts:
                    if part.text:
                        final_response_text += part.text + "\n"
        if not final_response_text: # Caso o agente não retorne nada em final_response
             print(f"⚠️ Agente {agent.name} não retornou uma resposta final explícita. Verifique as instruções do agente.")
             # Tentar obter a última mensagem do agente, se houver
             last_message = session_service.get_session(session_id="session_ALURA").get_last_agent_message()
             if last_message:
                 for part in last_message.parts:
                     if part.text:
                         final_response_text += part.text + "\n"

    except Exception as e:
        print(f"Erro ao executar o agente {agent.name}: {e}")
        return f"Erro ao processar com o agente {agent.name}."

    print(f"✅ Agente {agent.name} concluiu.")
    return final_response_text.strip()


def to_markdown_custom(text):
  """Formata o texto para exibição em Markdown no Colab."""
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def obter_resumo_ideias_existentes(banco_ideias):
    """Cria um resumo das ideias existentes para o Agente Auxiliador."""
    if not banco_ideias:
        return "Nenhuma ideia registrada anteriormente."

    resumos = ["Histórico de Ideias Registradas (para análise de similaridade):\n"]
    for ideia in banco_ideias:
        # Tentamos pegar as versões aprimoradas, se não, as originais
        problema_resumo = ideia.get('problema_aprimorado', ideia.get('problema_original', 'N/D'))
        solucao_resumo = ideia.get('solucao_aprimorada', ideia.get('solucao_original', 'N/D'))
        resumos.append(f"- ID: {ideia['id']}, Problema: {textwrap.shorten(problema_resumo, width=100)}, Solução: {textwrap.shorten(solucao_resumo, width=100)}")
    return "\n".join(resumos)

# Passo 4: Definição dos Agentes de IA

# --- Agente 1: O AUXILIADOR (Editor e Classificador Inicial) ---
agente_auxiliador = Agent(
    name="AgenteAuxiliadorALURA",
    model=MODEL_FOR_AGENTS, # Modelo Gemini
    instruction="""
    Você é um especialista em comunicação corporativa e gestão de ideias Kaizen, parte do sistema ALURA.
    Sua função é ser o "AUXILIADOR". Você receberá:
    1. Dados do funcionário (nome, matrícula, setor onde trabalha, local do problema).
    2. A descrição bruta de um problema e uma solução proposta por ele.
    3. Um histórico de ideias já registradas (se houver).

    Suas tarefas são:
    1.  REESCREVER E ORGANIZAR elaborando mais a ideia que foi dada pelo funcionário: Transforme a descrição do problema e da solução em textos mais objetivos, formais, claros e concisos, mantendo o significado original. Use linguagem profissional.
    2.  CHECAR HISTÓRICO E IDENTIFICAR SIMILARIDADES: Compare a ideia reescrita com o resumo das ideias que já foram registradas.
        - Se encontrar ideias muito parecidas (semanticamente, não apenas por palavras-chave), anote os IDs das ideias antigas que são similares e explique brevemente a similaridade.
        - Se a ideia parecer inédita ou substancialmente diferente das existentes, declare isso.
    3.  CONSOLIDAR DADOS: Apresente as informações básicas do funcionário.

    Formate TODA a sua SAÍDA EXATAMENTE assim, sem adicionar nenhum texto introdutório ou conclusivo fora desta estrutura:
    ```text
    **DADOS DO FUNCIONÁRIO (Confirmados):**
    Nome: [Nome do Funcionário]
    Matrícula: [Matrícula]
    Setor do Funcionário: [Setor onde o funcionário trabalha]
    Local Específico do Problema/Ideia: [Local informado]

    **IDEIA APRIMORADA (Pelo Agente Auxiliador):**
    Problema Identificado: [Problema reescrito por você de forma clara e profissional]
    Solução Proposta: [Solução reescrita por você de forma clara e profissional]

    **ANÁLISE DE SIMILARIDADE COM HISTÓRICO:**
    [Seu parecer: "Ideia parece inédita." OU "Atenção: Esta ideia apresenta similaridades com as seguintes ideias já registradas: ID XXX (Motivo: ...), ID YYY (Motivo: ...)."]
    ```
    """,
    description="Agente que refina a ideia do funcionário, verifica similaridades e consolida dados."
    # tools=[Google Search] # Opcional, se precisar de busca externa para enriquecer
)

# --- Agente 2: O TÉCNICO (Analista e Direcionador) ---
agente_tecnico = Agent(
    name="AgenteTecnicoALURA",
    model=MODEL_FOR_AGENTS,
    instruction="""
    Você é um analista técnico e de processos experiente em projetos de melhoria contínua (Kaizen), parte do sistema ALURA.
    Sua função é ser o "TÉCNICO". Você receberá a "IDEIA APRIMORADA" (problema e solução) que já foi processada pelo Agente Auxiliador.

    Suas tarefas são:
    1.  FAZER NOTAS TÉCNICAS: Avalie a ideia sob um ponto de vista prático e técnico.
        - Viabilidade Preliminar: A ideia parece ser implementável? (Ex: Alta, Média, Baixa, Requer Análise Detalhada).
        - Complexidade Estimada: Qual a complexidade para implementar? (Ex: Baixa, Média, Alta).
        - Recursos Chave Potenciais: Que tipos de recursos principais podem ser necessários? (Ex: Treinamento, Pequeno ajuste em software, Aquisição de material simples, Mudança de procedimento, Desenvolvimento complexo, Equipamento específico, etc.). Liste brevemente 1-3 tipos, se aplicável.
    2.  SUGERIR DIRECIONAMENTO INTERNO: Com base na natureza do problema/solução e no local do problema, sugira qual departamento ou função dentro de uma empresa (ex: Manutenção, TI, RH, Gestor do Setor da Ideia, Qualidade, Engenharia de Processos, Segurança do Trabalho, Comitê Kaizen) seria o mais indicado para uma avaliação mais aprofundada e possível implementação.

    Formate TODA a sua SAÍDA EXATAMENTE assim, sem adicionar nenhum texto introdutório ou conclusivo fora desta estrutura:
    ```text
    **ANÁLISE TÉCNICA PRELIMINAR (Pelo Agente Técnico):**
    Viabilidade Preliminar: [Sua avaliação. Ex: Média]
    Complexidade Estimada: [Sua avaliação. Ex: Baixa a Média]
    Recursos Chave Potenciais (Sugestão): [Sua lista breve. Ex: Treinamento da equipe, Ajuste de layout]

    **SUGESTÃO DE DIRECIONAMENTO INTERNO:**
    Departamento/Função Indicada para Avaliação: [Sua sugestão. Ex: Qualidade e Gestor do Setor XYZ]
    ```
    """,
    description="Agente que avalia a viabilidade técnica da ideia e sugere direcionamento."
    # tools=[Google Search] # Opcional, para pesquisar tecnologias ou soluções similares
)

# --- Agente 3: O REVISOR (Confirmador Final e Gerador de Relatório) ---
agente_revisor = Agent(
    name="AgenteRevisorALURA",
    model=MODEL_FOR_AGENTS, # Pode usar um modelo mais robusto se necessário para formatação complexa
    instruction="""
    Você é o Coordenador Final do processo de registro de ideias Kaizen no sistema ALURA.
    Sua função é ser o "REVISOR". Você receberá:
    1.  Os "DADOS DO FUNCIONÁRIO (Confirmados)" e a "IDEIA APRIMORADA" (do Agente Auxiliador).
    2.  A "ANÁLISE DE SIMILARIDADE COM HISTÓRICO" (do Agente Auxiliador).
    3.  A "ANÁLISE TÉCNICA PRELIMINAR" e a "SUGESTÃO DE DIRECIONAMENTO INTERNO" (do Agente Técnico).

    Suas tarefas são:
    1.  REVISAR O PROCESSO: Analise todas as informações coletadas e geradas pelos agentes anteriores para garantir consistência e completude para um registro inicial.
    2.  DEFINIR STATUS INICIAL: Com base na análise, defina um status inicial para a ideia. Use "Pronta para Avaliação Detalhada pelo(a) [Departamento/Função Sugerida pelo Agente Técnico]" se tudo estiver claro. Se a ideia for muito similar a uma já existente e não apresentar grande diferencial, ou se a análise técnica apontar baixa viabilidade imediata, você pode sugerir "Registrada - Requer Análise Crítica de Duplicidade/Viabilidade".
    3.  ADICIONAR NOTA FINAL: Inclua uma breve nota de fechamento ou observação (máximo 2 frases). Ex: "Ideia registrada com sucesso. Agradecemos a contribuição!" ou "Ideia registrada. Será encaminhada para análise da equipe XYZ."
    4.  GERAR RELATÓRIO FINAL CONSOLIDADO: Compile todas as informações em um relatório único e bem formatado.
        Use a data e hora atuais para o registro. O ID da Ideia será preenchido pelo sistema principal posteriormente.

    Formate TODA a sua SAÍDA (que será o relatório final) EXATAMENTE assim:
    ```text
    --- RELATÓRIO DA IDEIA KAIZEN (ALURA) ---
    ==================================================
    ID da Ideia: {{ID_DA_IDEIA_A_SER_PREENCHIDO}}
    Data e Hora do Registro: {{{{DATA_HORA_ATUAL}}}}

    --------------------------------------------------
    SOBRE O PROPONENTE:
      Nome: [Nome do Funcionário]
      Matrícula: [Matrícula]
      Setor do Funcionário: [Setor do Funcionário]
      Local Específico do Problema/Ideia: [Local do Problema/Ideia]

    --------------------------------------------------
    DESCRIÇÃO DA IDEIA (Refinada pelo Agente Auxiliador):
      Problema Identificado:
      [Problema Identificado, conforme processado pelo Agente Auxiliador]

      Solução Proposta:
      [Solução Proposta, conforme processada pelo Agente Auxiliador]

    --------------------------------------------------
    ANÁLISE PRELIMINAR AUTOMATIZADA (ALURA):

      1. Análise de Similaridade (Agente Auxiliador):
         [Resultado da Análise de Similaridade, conforme Agente Auxiliador]

      2. Análise Técnica (Agente Técnico):
         Viabilidade Preliminar: [Viabilidade, conforme Agente Técnico]
         Complexidade Estimada: [Complexidade, conforme Agente Técnico]
         Recursos Chave Potenciais: [Recursos, conforme Agente Técnico]

      3. Sugestão de Direcionamento (Agente Técnico):
         Departamento/Função Indicada: [Direcionamento, conforme Agente Técnico]

    --------------------------------------------------
    CONCLUSÃO E STATUS INICIAL (Pelo Agente Revisor):
      Status da Ideia: [Status definido por você, Agente Revisor]
      Nota do Revisor: [Sua nota final aqui]
    ==================================================
    --- FIM DO RELATÓRIO ---
    ```
    """,
    description="Agente que revisa todo o processo, define status e gera o relatório final da ideia."
)


# Passo 5: Função Principal de Processamento da Ideia com Agentes

def processar_nova_ideia_com_agentes(dados_funcionario, ideia_bruta, historico_ideias_formatado):
    """
    Orquestra a passagem da ideia pelos três agentes.
    Retorna o relatório final gerado pelo Agente Revisor e os dados estruturados.
    """
    print("\n🚀 Iniciando processamento da ideia com a equipe de Agentes ALURA...\n")

    # --- Agente 1: AUXILIADOR ---
    prompt_agente1 = f"""
    Dados do Funcionário:
    Nome: {dados_funcionario['nome']}
    Matrícula: {dados_funcionario['matricula']}
    Setor do Funcionário: {dados_funcionario['setor_funcionario']}
    Local Específico do Problema/Ideia: {dados_funcionario['local_problema']}

    Ideia Bruta:
    Problema: {ideia_bruta['problema']}
    Solução: {ideia_bruta['solucao']}

    {historico_ideias_formatado}
    """
    resposta_agente1_texto = call_agent_sync(agente_auxiliador, prompt_agente1)
    if not resposta_agente1_texto or "Erro ao processar" in resposta_agente1_texto:
        print("❌ Falha no processamento pelo Agente Auxiliador. Abortando.")
        return None, None
    display(Markdown("**Resposta do Agente Auxiliador:**"))
    display(to_markdown_custom(resposta_agente1_texto))

    # --- Agente 2: TÉCNICO ---
    # Precisamos extrair a "IDEIA APRIMORADA" da resposta do Agente 1 para passar ao Agente 2.
    # Isso é um ponto frágil se o Agente 1 não seguir o formato à risca.
    # Uma abordagem mais robusta seria o Agente 1 retornar um JSON, mas para manter o fluxo de texto:
    try:
        # Tentativa simples de extrair a seção da ideia aprimorada
        inicio_ideia_aprimorada = resposta_agente1_texto.find("**IDEIA APRIMORADA (Pelo Agente Auxiliador):**")
        fim_ideia_aprimorada = resposta_agente1_texto.find("**ANÁLISE DE SIMILARIDADE COM HISTÓRICO:**")
        if inicio_ideia_aprimorada != -1 and fim_ideia_aprimorada != -1:
            ideia_aprimorada_texto = resposta_agente1_texto[inicio_ideia_aprimorada:fim_ideia_aprimorada].strip()
        else: # Fallback se a extração falhar
            ideia_aprimorada_texto = "Não foi possível extrair a ideia aprimorada automaticamente. Usando a resposta completa do Agente 1." + \
                                     resposta_agente1_texto # Passa tudo como contexto
            print("⚠️ Não foi possível extrair a seção 'IDEIA APRIMORADA' da resposta do Agente 1. Passando a resposta completa para o Agente 2.")
    except Exception as e:
        print(f"Erro ao tentar extrair Ideia Aprimorada para Agente 2: {e}")
        ideia_aprimorada_texto = resposta_agente1_texto # Passa tudo

    prompt_agente2 = f"""
    Considerando a seguinte ideia já processada pelo Agente Auxiliador:
    {ideia_aprimorada_texto}

    Por favor, realize sua análise técnica e sugira o direcionamento.
    """
    resposta_agente2_texto = call_agent_sync(agente_tecnico, prompt_agente2)
    if not resposta_agente2_texto or "Erro ao processar" in resposta_agente2_texto:
        print("❌ Falha no processamento pelo Agente Técnico. Abortando.")
        return None, None
    display(Markdown("**Resposta do Agente Técnico:**"))
    display(to_markdown_custom(resposta_agente2_texto))


    # --- Agente 3: REVISOR ---
    data_hora_atual_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    prompt_agente3 = f"""
    INFORMAÇÕES CONSOLIDADAS PARA REVISÃO FINAL:

    {resposta_agente1_texto}

    {resposta_agente2_texto}

    INSTRUÇÃO ADICIONAL:
    Por favor, use "{data_hora_atual_str}" como a DATA_HORA_ATUAL no relatório.
    O ID da Ideia será preenchido depois, deixe como "{{ID_DA_IDEIA_A_SER_PREENCHIDO}}".
    Gere o relatório final completo conforme suas instruções.
    """
    relatorio_final_texto = call_agent_sync(agente_revisor, prompt_agente3)
    if not relatorio_final_texto or "Erro ao processar" in relatorio_final_texto:
        print("❌ Falha no processamento pelo Agente Revisor. Abortando.")
        return None, None

    print("\n🎉 Processamento da ideia concluído pelos agentes ALURA! 🎉")

    # Armazenar dados estruturados (um desafio parsear tudo do texto, simplificação abaixo)
    # Idealmente, os agentes retornariam JSONs ou teríamos parsing mais robusto.
    # Por ora, focaremos no relatório textual.
    dados_consolidados = {
        "dados_funcionario": dados_funcionario,
        "ideia_bruta": ideia_bruta,
        "resposta_agente1": resposta_agente1_texto,
        "resposta_agente2": resposta_agente2_texto,
        "relatorio_final": relatorio_final_texto, # Gerado pelo Agente 3
        "data_registro_sistema": data_hora_atual_str
    }

    return relatorio_final_texto, dados_consolidados


def exibir_banco_de_ideias_processadas_debug():
    """Exibe o conteúdo atual do banco de ideias processadas para depuração."""
    print("\n--- Estado Atual do Banco de Ideias Processadas (ALURA - DEBUG) ---")
    if not banco_de_ideias_processadas:
        print("Nenhuma ideia processada e registrada ainda.")
    else:
        for idx, ideia_reg in enumerate(banco_de_ideias_processadas):
            print(f"\n--- Ideia Registrada #{idx + 1} (ID Interno: {ideia_reg.get('id', 'N/A')}) ---")
            print(f"Funcionário: {ideia_reg['dados_funcionario']['nome']}")
            print(f"Data Registro (Sistema): {ideia_reg['data_registro_sistema']}")
            # print(f"Relatório Completo:\n{ideia_reg['relatorio_final']}") # Pode ser muito longo
            print("Visualizar relatório completo ao final do registro individual.")
            print("-" * 20)
    print("--- Fim do Banco de Ideias (DEBUG) ---\n")

# Passo 6: Fluxo Principal do Programa ALURA

if __name__ == "__main__":
    # Verificar se a API Key está configurada antes de prosseguir
    if "GOOGLE_API_KEY" not in os.environ or not os.environ["GOOGLE_API_KEY"]:
        print("ERRO CRÍTICO: A API Key do Google não está configurada.")
        print("Por favor, configure a variável de ambiente GOOGLE_API_KEY ou use o Colab Secrets.")
        # Interrompe a execução se a chave não estiver presente.
        # Em um script real, você poderia usar exit() aqui. No Colab, a célula pode não continuar.
    else:
        print("Configuração da API Key verificada. Prosseguindo com o ALURA.")

        id_sequencial = 0
        while True:
            info_funcionario = iniciar_chat_coleta_funcionario()
            ideia_original = coletar_ideia_bruta()

            resumo_historico = obter_resumo_ideias_existentes(banco_de_ideias_processadas)

            relatorio_gerado, dados_da_ideia_processada = processar_nova_ideia_com_agentes(
                info_funcionario,
                ideia_original,
                resumo_historico
            )

            if relatorio_gerado and dados_da_ideia_processada:
                id_sequencial += 1
                dados_da_ideia_processada["id"] = id_sequencial # ID sequencial simples

                # Substituir placeholders no relatório final
                relatorio_final_com_id_e_data = relatorio_gerado.replace("{{ID_DA_IDEIA_A_SER_PREENCHIDO}}", str(id_sequencial))
                # A data/hora já é inserida pelo prompt do Agente 3, mas caso precise de um replace:
                # relatorio_final_com_id_e_data = relatorio_final_com_id_e_data.replace("{{DATA_HORA_ATUAL}}", dados_da_ideia_processada["data_registro_sistema"])

                dados_da_ideia_processada["relatorio_final"] = relatorio_final_com_id_e_data
                banco_de_ideias_processadas.append(dados_da_ideia_processada)

                print("\n\n📋 --- RELATÓRIO FINAL DA IDEIA KAIZEN (Gerado pelo ALURA) --- 📋")
                # Exibe o relatório formatado como Markdown para melhor visualização no Colab
                display(Markdown(relatorio_final_com_id_e_data))
                print("\nVocê pode tirar um print desta tela ou copiar o texto do relatório para seus registros.")
                print("--------------------------------------------------------------------")
            else:
                print("\n❌ Não foi possível processar e registrar a ideia devido a erros anteriores.")

            # Opcional: Mostrar o banco de ideias após cada registro durante o desenvolvimento
            # exibir_banco_de_ideias_processadas_debug()

            while True:
                continuar = input("\n\nDeseja registrar outra ideia Kaizen com o ALURA? (s/n): ").lower().strip()
                if continuar in ['s', 'n']:
                    break
                else:
                    print("Opção inválida. Digite 's' para sim ou 'n' para não.")

            if continuar == 'n':
                print("\nObrigado por usar o ALURA! Suas ideias são muito importantes para a melhoria contínua.")
                exibir_banco_de_ideias_processadas_debug() # Mostra todas as ideias no final
                break
            print("\n" + "="*60 + "\n")

API Key carregada a partir do Colab userdata.
Configuração da API Key verificada. Prosseguindo com o ALURA.
******************************************
* ALURA *
* (Sistema de Análise e *
* Desenvolvimento de Aprimoramentos) *
******************************************


Bem-vindo ao ALURA! O seu Sistema Inteligente para Ideias Kaizen.
Vamos registrar sua ideia para torná-la ainda melhor!



KeyboardInterrupt: Interrupted by user

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# -*- coding: utf-8 -*-
# Importações necessárias
%pip -q install google-genai google-generativeai==0.5.0 --upgrade # Instala a SDK e o ADK
%pip install google-adk --upgrade # Garante que o ADK está atualizado

import os
from google.colab import userdata
from google.adk.agents import Agent # Importa a classe Agent do ADK
from google.adk.runners import Runner # Importa a classe Runner do ADK
from google.adk.sessions import InMemorySessionService # Importa o serviço de sessão em memória
from google.genai import types # Para criar conteúdos (Content e Part)
from google.genai import client as genai_client # Importa o cliente genai explicitamente
from datetime import datetime # Para data e hora
import textwrap # Para formatar texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import warnings # Para gerenciar avisos

warnings.filterwarnings("ignore") # Ignora alguns avisos

# Configura a API Key do Google Gemini
# É importante que você tenha adicionado sua GOOGLE_API_KEY nos Segredos do Colab
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    # Inicializa o cliente GenAI
    genai = genai_client.Client()
except Exception as e:
    print(f"Erro ao configurar a API Key ou inicializar o cliente GenAI: {e}")
    print("Por favor, verifique se sua 'GOOGLE_API_KEY' está configurada corretamente nos Segredos do Colab.")
    # Você pode querer sair ou lidar com o erro de outra forma se a chave for essencial.

# --- Simulação de um banco de dados em memória ---
# Lista para armazenar as ideias registradas
banco_de_ideias = []
# Contador para gerar IDs únicos para as ideias
idea_counter = 0

# --- Funções Auxiliares ---

def mostrar_logo():
    """Exibe o logotipo textual do SADA."""
    print("*"*38)
    print("* S A D A *")
    print("* (Sistema de Análise e *")
    print("* Desenvolvimento de Aprimoramentos) *")
    print("*"*38)
    print("\n")

def iniciar_chat():
    """Inicia a conversa e coleta dados iniciais do funcionário."""
    mostrar_logo()
    print("Bem-vindo ao SADA!")
    print("Estou aqui para te ajudar a registrar sua ideia Kaizen.\n")

    while True:
        nome = input("Para começarmos, por favor, me informe seu nome completo: ").strip()
        if nome:
            break
        else:
            print("Nome inválido. Por favor, digite seu nome.")

    while True:
        matricula = input(f"Olá, {nome}! Agora, por favor, informe sua matrícula (apenas números): ").strip()
        if matricula.isdigit():
            break
        else:
            print("Matrícula inválida. Por favor, digite apenas números.")

    while True:
        setor_ideia = input(f"Obrigado, {nome}! Em qual setor você trabalha ou para qual setor é a sua ideia? ").strip()
        if setor_ideia:
            break
        else:
            print("Setor inválido. Por favor, digite o setor.")

    return nome, matricula, setor_ideia

def coletar_ideia():
    """Coleta a descrição do problema e da solução proposta."""
    print("\n--- Detalhes da Ideia Kaizen ---")
    while True:
        problema = input("Descreva a ideia Kaizen que você tem. Qual é o problema ou oportunidade de melhoria que você identificou?\n").strip()
        if problema:
            break
        else:
            print("A descrição do problema não pode estar vazia.")

    while True:
        solucao = input("Interessante! E qual seria a sua proposta de solução para este problema/oportunidade?\n").strip()
        if solucao:
            break
        else:
            print("A descrição da solução não pode estar vazia.")
    return problema, solucao

def call_agent(agent: Agent, message_text: str) -> str:
    """
    Função auxiliar para executar um agente com uma mensagem e retornar a resposta final.
    Adaptada da sua função.
    """
    try:
        # Cria um serviço de sessão em memória para esta execução
        session_service = InMemorySessionService()
        # Cria uma nova sessão simples
        session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="temp_session")
        # Cria um Runner para o agente
        runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
        # Cria o conteúdo da mensagem de entrada
        content = types.Content(role="user", parts=[types.Part(text=message_text)])

        final_response = ""
        # Itera assincronamente pelos eventos retornados durante a execução do agente
        for event in runner.run(user_id="user1", session_id="temp_session", new_message=content):
            if event.is_final_response():
                for part in event.content.parts:
                    if part.text is not None:
                        final_response += part.text
                        # final_response += "\n" # Adiciona newline, opcional dependendo do formato desejado

        return final_response.strip() # Retorna a resposta final, removendo espaços em branco extras
    except Exception as e:
        print(f"Erro ao executar o agente {agent.name}: {e}")
        return f"Erro interno ao processar com o Agente {agent.name}."


def get_existing_ideas_summary():
    """Prepara um resumo das ideias existentes para o Agente AUXILIADOR."""
    summaries = []
    # Itera sobre uma cópia para evitar problemas se a lista for modificada
    for ideia in list(banco_de_ideias):
        # Limita o tamanho para não estourar o contexto do modelo
        problema_snippet = ideia.get('problema_bruto', 'N/A')[:100] + ('...' if len(ideia.get('problema_bruto', '')) > 100 else '')
        solucao_snippet = ideia.get('solucao_bruta', 'N/A')[:100] + ('...' if len(ideia.get('solucao_bruta', '')) > 100 else '')
        summaries.append(f"ID: {ideia.get('id', 'N/A')} - Problema: {problema_snippet} | Solução: {solucao_snippet} | Status: {ideia.get('status', 'N/A')}")

    return "\n".join(summaries) if summaries else "Nenhuma ideia registrada anteriormente no banco de dados."

# --- Definição e Chamada dos Agentes ---

def call_auxiliador_agent(nome, matricula, setor, problema_bruto, solucao_bruta, existing_ideas_summary):
    """Chama o Agente AUXILIADOR para processar a ideia bruta e checar similaridades."""
    auxiliador = Agent(
        name="Agente_Auxiliador",
        model="gemini-2.0-flash", # Pode usar flash para custo/velocidade
        instruction=f"""
        Você é o AUXILIADOR, especialista em processar e classificar ideias Kaizen submetidas por funcionários.
        Sua função é:
        1. Transformar a descrição informal do PROBLEMA e da SOLUÇÃO em um texto claro, profissional e objetivo.
        2. Comparar a essência da nova ideia (problema e solução) com os RESUMOS DE IDEIAS EXISTENTES fornecidos.
        3. Identificar os IDs das ideias existentes que parecem SIMILARES ou CONFLITANTES com a nova proposta.
        4. Organizar as informações do funcionário.

        Sua resposta deve ser estruturada claramente, começando com a versão aprimorada da ideia, seguida pela lista de similaridades encontradas (ou uma indicação se nenhuma foi encontrada) e os dados do funcionário. Use o seguinte formato de saída:

        [IDEIA APRIMORADA]
        Problema Aprimorado: [Sua reescrita clara do problema]
        Solução Aprimorada: [Sua reescrita clara da solução]

        [ANÁLISE DE SIMILARIDADES]
        Similaridades Encontradas (IDs): [Liste os IDs das ideias similares, separados por vírgula, ou escreva 'Nenhuma similaridade clara encontrada.']
        Nota sobre Similaridades: [Uma breve nota se necessário, por exemplo, se a ideia parece complementar em vez de conflitante]

        [DADOS DO FUNCIONÁRIO]
        Nome: [Nome do funcionário]
        Matrícula: [Matrícula]
        Setor (Ideia): [Setor]

        Aqui estão as informações que você deve processar:

        DADOS DO FUNCIONÁRIO:
        Nome: {nome}
        Matrícula: {matricula}
        Setor: {setor}

        IDEIA BRUTA:
        Problema: {problema_bruto}
        Solução: {solucao_bruta}

        RESUMOS DE IDEIAS EXISTENTES NO BANCO DE DADOS PARA COMPARAÇÃO:
        {existing_ideas_summary}

        Comece a resposta com [IDEIA APRIMORADA].
        """,
        description="Agente que processa e classifica ideias Kaizen, identificando similaridades."
        # Ferramentas não são necessárias para este agente, pois ele opera sobre o texto e o resumo fornecidos.
    )
    print("Auxiliador processando...")
    return call_agent(auxiliador, "") # A instrução já contém todos os dados, passamos mensagem vazia.

def call_tecnico_agent(auxiliador_output, setor_da_ideia):
    """Chama o Agente TÉCNICO para analisar a viabilidade e sugerir um responsável."""
    tecnico = Agent(
        name="Agente_Tecnico",
        model="gemini-2.0-flash", # Pode usar flash
        instruction=f"""
        Você é o TÉCNICO, especialista em fazer uma análise inicial de viabilidade e direcionamento para ideias Kaizen.
        Sua função é:
        1. Ler a 'IDEIA APRIMORADA' e os 'DADOS DO FUNCIONÁRIO' fornecidos pela análise do AUXILIADOR.
        2. Fazer uma breve 'Nota Técnica Preliminar' sobre a aparente complexidade, viabilidade ou recursos que podem ser necessários (baseado na descrição). Seja conciso.
        3. Sugerir o 'Departamento ou Responsável Interno' mais provável para avaliar a ideia com base no PROBLEMA APRIMORADO, SOLUÇÃO APRIMORADA e, crucialmente, no SETOR DA IDEIA. Use lógica comum da empresa:
           - Problemas em máquinas/equipamentos -> Manutenção
           - Problemas em processos/fluxos/qualidade -> Qualidade ou Engenharia de Processos
           - Problemas relacionados a pessoas/treinamento/equipes -> RH ou Gestor do Setor
           - Problemas de infraestrutura/prédio -> Manutenção ou Administração
           - Problemas específicos de um setor -> Gestor do Setor
        4. Sua resposta deve ser estruturada, fornecendo apenas as notas técnicas e a sugestão de responsável. Use o seguinte formato de saída:

        [ANÁLISE TÉCNICA PRELIMINAR]
        Nota Técnica Preliminar: [Sua nota concisa]
        Sugestão de Responsável: [Departamento ou Cargo/Função sugerido]

        Aqui estão as informações fornecidas pelo Agente AUXILIADOR e o Setor da Ideia:

        {auxiliador_output}

        Setor da Ideia: {setor_da_ideia}

        Comece a resposta com [ANÁLISE TÉCNICA PRELIMINAR].
        """,
        description="Agente que analisa a viabilidade inicial e sugere o responsável interno pela ideia."
    )
    print("Técnico processando...")
    return call_agent(tecnico, "") # A instrução já contém todos os dados.

def call_revisor_agent(idea_id, data_registro, nome, matricula, setor, auxiliador_output, tecnico_output):
    """Chama o Agente REVISOR para consolidar e gerar o relatório final."""
    revisor = Agent(
        name="Agente_Revisor",
        model="gemini-2.0-flash", # Pode usar flash
         instruction=f"""
        Você é o REVISOR, o responsável por consolidar todas as informações processadas pelos agentes anteriores e gerar o RELATÓRIO FINAL formal da ideia Kaizen.
        Sua função é:
        1. Coletar e organizar as informações do 'AUXILIADOR' e do 'TÉCNICO', juntamente com os dados básicos da ideia (ID, Data, Funcionário).
        2. Definir o 'STATUS ATUAL DA IDEIA' inicial como "Pronto para Avaliação Técnica".
        3. Formatar todas essas informações em um relatório claro, organizado e fácil de ler, ideal para ser "printado" (capturado a tela).
        4. Incluir o logotipo textual "SADA" no topo do relatório.

        O RELATÓRIO FINAL deve seguir EXATAMENTE este formato:

        --- RELATÓRIO DA IDEIA KAİZEN ---
        * S A D A *
        * (Sistema de Análise e *
        * Desenvolvimento de Aprimoramentos) *
        -----------------------------------
        ID da Ideia: {idea_id}
        Data do Registro: {data_registro}
        -----------------------------------
        FUNCIONÁRIO:
        Nome: {nome}
        Matrícula: {matricula}
        Setor (Ideia): {setor}
        -----------------------------------
        DETALHES DA IDEIA:
        [Conteúdo da seção 'IDEIA APRIMORADA' do Auxiliador, formatado com marcadores se necessário]
        -----------------------------------
        ANÁLISE INICIAL DOS AGENTES:
        [Conteúdo da seção 'ANÁLISE DE SIMILARIDADES' do Auxiliador]

        [Conteúdo da seção 'ANÁLISE TÉCNICA PRELIMINAR' do Técnico]
        -----------------------------------
        STATUS ATUAL DA IDEIA:
        Status: Pronto para Avaliação Técnica
        -----------------------------------
        --- FIM DO RELATÓRIO ---

        Aqui estão as informações dos agentes anteriores que você deve usar para preencher o relatório:

        Informações do AUXILIADOR:
        {auxiliador_output}

        Informações do TÉCNICO:
        {tecnico_output}

        Sua resposta deve ser APENAS o texto do RELATÓRIO FINAL formatado como especificado acima.
        """,
        description="Agente que consolida as análises e gera o relatório final da ideia Kaizen."
    )
    print("Revisor processando...")
    return call_agent(revisor, "") # A instrução já contém todos os dados.


# --- Fluxo Principal do Programa ---
if __name__ == "__main__":
    while True:
        # 1. Coleta inicial de dados do funcionário e da ideia bruta
        nome_funcionario, matricula_funcionario, setor_da_ideia = iniciar_chat()
        problema_identificado, solucao_proposta = coletar_ideia()

        # --- Processamento da Ideia através da Sequência de Agentes ---
        print("\n--- Iniciando Processamento da Ideia com Agentes ---")

        # Obtém o resumo das ideias existentes para o Auxiliador
        existing_summary = get_existing_ideas_summary()

        # 2. Chama o Agente AUXILIADOR
        auxiliador_result_text = call_auxiliador_agent(
            nome=nome_funcionario,
            matricula=matricula_funcionario,
            setor=setor_da_ideia,
            problema_bruto=problema_identificado,
            solucao_bruta=solucao_proposta,
            existing_ideas_summary=existing_summary
        )
        print("✅ Auxiliador concluído.")
        # Opcional: print(f"\n--- Saída Auxiliador ---\n{auxiliador_result_text}\n------------------------\n") # Debug

        # 3. Chama o Agente TÉCNICO, passando o resultado do Auxiliador
        tecnico_result_text = call_tecnico_agent(
            auxiliador_output=auxiliador_result_text,
            setor_da_ideia=setor_da_ideia # Passamos o setor também, pois é relevante para o direcionamento
        )
        print("✅ Técnico concluído.")
        # Opcional: print(f"\n--- Saída Técnico ---\n{tecnico_result_text}\n----------------------\n") # Debug

        # 4. Prepara dados para o Agente REVISOR (ID, Data/Hora)
        global idea_counter # Usa a variável global
        idea_counter += 1 # Incrementa o contador para o novo ID
        current_idea_id = idea_counter
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # 5. Chama o Agente REVISOR, passando os resultados dos agentes anteriores e os dados finais
        revisor_final_report_text = call_revisor_agent(
            idea_id=current_idea_id,
            data_registro=timestamp,
            nome=nome_funcionario,
            matricula=matricula_funcionario,
            setor=setor_da_ideia,
            auxiliador_output=auxiliador_result_text,
            tecnico_output=tecnico_result_text
        )
        print("✅ Revisor concluído. Relatório gerado.")
        print("\n--- Fim do Processamento com Agentes ---\n")

        # --- Armazenamento e Exibição ---

        # 6. Armazena a ideia processada no banco de dados em memória
        # Armazenamos o relatório final gerado para fácil recuperação,
        # além dos dados brutos e as saídas dos agentes intermediários, se necessário.
        banco_de_ideias.append({
            "id": current_idea_id,
            "data_registro": timestamp,
            "nome_funcionario": nome_funcionario,
            "matricula": matricula_funcionario,
            "setor": setor_da_ideia,
            "problema_bruto": problema_identificado,
            "solucao_bruta": solucao_proposta,
            "auxiliador_output": auxiliador_result_text, # Opcional: guardar outputs intermediários
            "tecnico_output": tecnico_result_text,     # Opcional: guardar outputs intermediários
            "revisor_report_text": revisor_final_report_text, # O relatório final em texto
            "status": "Pronto para Avaliação Técnica" # Status definido pelo fluxo
        })

        # 7. Exibe o relatório final formatado para o usuário
        print("\n--- RELATÓRIO FINAL ---")
        # Usamos display(Markdown(...)) para renderizar o texto formatado no Colab
        display(Markdown(revisor_final_report_text))
        print("\nVocê pode tirar um print desta tela para seus registros.")
        print("-----------------------\n")


        # 8. Pergunta se o usuário deseja registrar outra ideia
        while True:
            continuar = input("Deseja registrar outra ideia? (s/n): ").lower().strip()
            if continuar in ['s', 'n']:
                break
            else:
                print("Opção inválida. Digite 's' para sim ou 'n' para não.")

        if continuar == 'n':
            print("\nObrigado por usar o SADA! Suas ideias são muito importantes.")
            # Opcional: Mostrar todas as ideias registradas no final (pode ser útil para debug)
            # print("\n--- Todas as Ideias Registradas (Resumo) ---")
            # if banco_de_ideias:
            #     for ideia in banco_de_ideias:
            #         print(f"ID: {ideia['id']} - {ideia['data_registro']} - {ideia['nome_funcionario']} ({ideia['matricula']}) - Status: {ideia['status']}")
            # else:
            #     print("Nenhuma ideia foi registrada nesta sessão.")
            # print("---------------------------------------------")
            break # Sai do loop principal

        print("\n" + "="*70 + "\n") # Separador para a próxima ideia

In [ ]:
# -*- coding: utf-8 -*-
# Importações necessárias
# Verifica e instala as bibliotecas se necessário
try:
    import google.genai
    import google.adk
except ImportError:
    print("Instalando bibliotecas necessárias...")
    %pip -q install google-genai google-generativeai==0.5.0 --upgrade
    %pip install google-adk --upgrade
    print("Instalação concluída.")
    import google.genai
    import google.adk


import os
from google.colab import userdata
from google.adk.agents import Agent # Importa a classe Agent do ADK
from google.adk.runners import Runner # Importa a classe Runner do ADK
from google.adk.sessions import InMemorySessionService # Importa o serviço de sessão em memória
from google.genai import types # Para criar conteúdos (Content e Part)
from google.genai import client as genai_client # Importa o cliente genai explicitamente
from datetime import datetime # Para data e hora
import textwrap # Para formatar texto (mantido, pode ser útil)
from IPython.display import display, Markdown, HTML # Importa HTML para centralizar
import warnings # Para gerenciar avisos

warnings.filterwarnings("ignore") # Ignora alguns avisos

# Configura a API Key do Google Gemini
# É importante que você tenha adicionado sua GOOGLE_API_KEY nos Segredos do Colab
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    # Inicializa o cliente GenAI
    genai = genai_client.Client()
    # Verifica se a chave funcionou tentando listar modelos
    genai.list_models()
except Exception as e:
    print(f"Erro ao configurar a API Key ou inicializar o cliente GenAI: {e}")
    print("Por favor, verifique se sua 'GOOGLE_API_KEY' está configurada corretamente nos Segredos do Colab (ícone de chave 🔑 na barra lateral esquerda).")
    print("Certifique-se de que a chave API do Gemini esteja lá e nomeada como 'GOOGLE_API_KEY'.")
    # Se a chave não funcionar, o script continuará, mas os agentes falharão.

# --- Simulação de um banco de dados em memória ---
# Lista para armazenar as ideias registradas
banco_de_ideias = []
# Contador para gerar IDs únicos para as ideias
idea_counter = 0

# --- Funções Auxiliares ---

def mostrar_logo():
    """Exibe o logotipo textual do SADA com formatação."""
    # Códigos ANSI para negrito (\033[1m) e reset (\033[0m)
    NEGRITO = "\033[1m"
    RESET = "\033[0m"

    print("\n") # Adiciona uma linha em branco em cima para "parecer" maior
    print(f"{NEGRITO}{'*'*60}{RESET}") # Negrito nas estrelas
    print(f"{NEGRITO} A L U R A {RESET}") # Espaçamento e negrito
    print(f"{NEGRITO}* (Sistema de Análise e ")
    print(f"* Desenvolvimento de Aprimoramentos) *{RESET}") # Negrito no texto, mas precisa resetar no final da linha formatada
    print(f"{NEGRITO}{'*'*60}{RESET}") # Negrito nas estrelas
    print("\n") # Adiciona uma linha em branco embaixo

def iniciar_chat():
    """Inicia a conversa e coleta dados iniciais do funcionário."""
    mostrar_logo()
    print("Bem-vindo ao SADA!")
    print("Estou aqui para te ajudar a registrar sua ideia Kaizen.\n")

    while True:
        nome = input("Para começarmos, por favor, me informe seu nome completo: ").strip()
        if nome:
            break
        else:
            print("Nome inválido. Por favor, digite seu nome.")

    while True:
        matricula = input(f"Olá, {nome}! Agora, por favor, informe sua matrícula (apenas números): ").strip()
        # Permite matrícula vazia, mas valida se tiver algo
        if not matricula or matricula.isdigit():
             break
        else:
            print("Matrícula inválida. Por favor, digite apenas números ou deixe em branco.")


    while True:
        setor_ideia = input(f"Obrigado, {nome}! Em qual setor você trabalha ou para qual setor é a sua ideia? ").strip()
        if setor_ideia:
            break
        else:
            print("Setor inválido. Por favor, digite o setor.")

    return nome, matricula, setor_ideia

def coletar_ideia():
    """Coleta a descrição do problema e da solução proposta."""
    print("\n--- Detalhes da Ideia Kaizen ---")
    while True:
        problema = input("Descreva a ideia Kaizen que você tem. Qual é o problema ou oportunidade de melhoria que você identificou?\n").strip()
        if problema:
            break
        else:
            print("A descrição do problema não pode estar vazia.")

    while True:
        solucao = input("Interessante! E qual seria a sua proposta de solução para este problema/oportunidade?\n").strip()
        if solucao:
            break
        else:
            print("A descrição da solução não pode estar vazia.")
    return problema, solucao

def call_agent(agent: Agent, message_text: str = "") -> str:
    """
    Função auxiliar para executar um agente com uma mensagem e retornar a resposta final.
    A mensagem padrão é vazia, pois a instrução já contém os dados.
    """
    try:
        # Cria um serviço de sessão em memória para esta execução
        session_service = InMemorySessionService()
        # Cria uma nova sessão simples
        session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="temp_session")
        # Cria um Runner para o agente
        runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
        # Cria o conteúdo da mensagem de entrada
        content = types.Content(role="user", parts=[types.Part(text=message_text)])

        final_response = ""
        # Itera assincronamente pelos eventos retornados durante a execução do agente
        for event in runner.run(user_id="user1", session_id="temp_session", new_message=content):
            if event.is_final_response():
                for part in event.content.parts:
                    if part.text is not None:
                        final_response += part.text
                        # final_response += "\n" # Adiciona newline, opcional dependendo do formato desejado

        return final_response.strip() # Retorna a resposta final, removendo espaços em branco extras
    except Exception as e:
        print(f"Erro ao executar o agente {agent.name}: {e}")
        # Retorna uma mensagem de erro clara se o agente falhar
        return f"[ERRO NO AGENTE {agent.name}: {e}]"


def get_existing_ideas_summary():
    """Prepara um resumo das ideias existentes para o Agente AUXILIADOR."""
    summaries = []
    # Itera sobre uma cópia para evitar problemas se a lista for modificada
    for ideia in list(banco_de_ideias):
        # Limita o tamanho para não estourar o contexto do modelo
        problema_snippet = ideia.get('problema_bruto', 'N/A')[:100] + ('...' if len(ideia.get('problema_bruto', '')) > 100 else '')
        solucao_snippet = ideia.get('solucao_bruta', 'N/A')[:100] + ('...' if len(ideia.get('solucao_bruta', '')) > 100 else '')
        # Inclui o status para ajudar o agente a diferenciar ideias ativas/fechadas, etc.
        summaries.append(f"ID: {ideia.get('id', 'N/A')} | Status: {ideia.get('status', 'N/A')} | Problema: {problema_snippet} | Solução: {solucao_snippet}")

    return "\n".join(summaries) if summaries else "Nenhuma ideia registrada anteriormente no banco de dados."

# --- Definição e Chamada dos Agentes ---

def call_auxiliador_agent(nome, matricula, setor, problema_bruto, solucao_bruta, existing_ideas_summary):
    """Chama o Agente AUXILIADOR para processar a ideia bruta e checar similaridades."""
    # Tenta obter a lista de modelos disponíveis para escolher um apropriado
    try:
        model_name = "gemini-1.5-flash-latest"
        # Você pode adicionar lógica aqui para escolher outro modelo se o flash não estiver disponível
        # available_models = [m.name for m in genai.list_models()]
        # if f"models/{model_name}" not in available_models:
        #     print(f"Modelo {model_name} não disponível. Usando outro modelo...")
        #     model_name = "gemini-pro" # Exemplo de fallback
    except Exception as e:
        print(f"Não foi possível listar modelos para o Auxiliador: {e}. Usando modelo padrão.")
        model_name = "gemini-pro" # Fallback se a API key não estiver configurada

    auxiliador = Agent(
        name="Agente_Auxiliador",
        model=model_name, # Usa o modelo definido
        instruction=f"""
        Você é o AUXILIADOR, especialista em processar e classificar ideias Kaizen submetidas por funcionários.
        Sua função é:
        1. Transformar a descrição informal do PROBLEMA e da SOLUÇÃO em um texto claro, profissional e objetivo.
        2. Comparar a essência da nova ideia (problema e solução) com os RESUMOS DE IDEIAS EXISTENTES fornecidos.
        3. Identificar os IDs das ideias existentes que parecem SIMILARES ou CONFLITANTES com a nova proposta. Mencione brevemente o problema/solução da ideia similar encontrada se possível.
        4. Organizar as informações do funcionário.

        Sua resposta deve ser estruturada claramente usando as tags [IDEIA APRIMORADA], [ANÁLISE DE SIMILARIDADES] e [DADOS DO FUNCIONÁRIO] para delimitar as seções. Dentro de cada seção, use quebras de linha e marcadores se apropriado para clareza.

        [IDEIA APRIMORADA]
        Problema Aprimorado: [Sua reescrita clara do problema]
        Solução Aprimorada: [Sua reescrita clara da solução]

        [ANÁLISE DE SIMILARIDADES]
        Similaridades Encontradas (IDs): [Liste os IDs das ideias similares e uma breve descrição (ex: ID 1 - Problema X...), separados por ponto e vírgula, ou escreva 'Nenhuma similaridade clara encontrada.']
        Nota sobre Similaridades: [Uma breve nota se necessário, por exemplo, se a ideia parece complementar em vez de conflitante ou se é uma repetição clara]

        [DADOS DO FUNCIONÁRIO]
        Nome: [Nome do funcionário]
        Matrícula: [Matrícula]
        Setor (Ideia): [Setor]

        Aqui estão as informações que você deve processar:

        DADOS DO FUNCIONÁRIO:
        Nome: {nome}
        Matrícula: {matricula}
        Setor: {setor}

        IDEIA BRUTA:
        Problema: {problema_bruto}
        Solução: {solucao_bruta}

        RESUMOS DE IDEIAS EXISTENTES NO BANCO DE DADOS PARA COMPARAÇÃO:
        {existing_ideas_summary}

        Comece sua resposta diretamente com a seção [IDEIA APRIMORADA].
        """,
        description="Agente que processa e classifica ideias Kaizen, identificando similaridades."
    )
    print("Auxiliador processando...")
    return call_agent(auxiliador) # Passa mensagem vazia, instrução contém tudo.

def call_tecnico_agent(auxiliador_output, setor_da_ideia):
    """Chama o Agente TÉCNICO para analisar a viabilidade e sugerir um responsável."""
    try:
        model_name = "gemini-1.5-flash-latest"
    except Exception as e:
        print(f"Não foi possível listar modelos para o Técnico: {e}. Usando modelo padrão.")
        model_name = "gemini-pro" # Fallback

    tecnico = Agent(
        name="Agente_Tecnico",
        model=model_name, # Usa o modelo definido
        instruction=f"""
        Você é o TÉCNICO, especialista em fazer uma análise inicial de viabilidade e direcionamento para ideias Kaizen.
        Sua função é:
        1. Ler a 'IDEIA APRIMORADA' fornecida na análise do AUXILIADOR. Ignore as seções [ANÁLISE DE SIMILARIDADES] e [DADOS DO FUNCIONÁRIO].
        2. Fazer uma breve 'Nota Técnica Preliminar' sobre a aparente complexidade, viabilidade ou recursos que podem ser necessários (baseado na descrição do problema e solução aprimorados). Seja conciso e direto.
        3. Sugerir o 'Departamento ou Responsável Interno' mais provável para avaliar a ideia com base no PROBLEMA APRIMORADO, SOLUÇÃO APRIMORADA e, crucialmente, no SETOR DA IDEIA. Use lógica comum da empresa:
           - Problemas em máquinas/equipamentos -> Manutenção
           - Problemas em processos/fluxos/qualidade -> Qualidade ou Engenharia de Processos
           - Problemas relacionados a pessoas/treinamento/equipes -> RH ou Gestor do Setor
           - Problemas de infraestrutura/prédio -> Manutenção ou Administração
           - Problemas de TI/Sistemas -> TI
           - Problemas específicos de um setor -> Gestor do Setor
           - Problemas gerais/administrativos -> Gestão/Administração
        4. Sua resposta deve ser estruturada usando a tag [ANÁLISE TÉCNICA PRELIMINAR] para delimitar a seção e marcadores/quebras de linha para clareza.

        [ANÁLISE TÉCNICA PRELIMINAR]
        Nota Técnica Preliminar: [Sua nota concisa]
        Sugestão de Responsável: [Departamento ou Cargo/Função sugerido]

        Aqui estão as informações fornecidas pelo Agente AUXILIADOR e o Setor da Ideia:

        {auxiliador_output}

        Setor da Ideia: {setor_da_ideia}

        Comece sua resposta diretamente com a seção [ANÁLISE TÉCNICA PRELIMINAR].
        """,
        description="Agente que analisa a viabilidade inicial e sugere o responsável interno pela ideia."
    )
    print("Técnico processando...")
    return call_agent(tecnico) # Passa mensagem vazia.

def call_revisor_agent(idea_id, data_registro, nome, matricula, setor, auxiliador_output, tecnico_output):
    """Chama o Agente REVISOR para consolidar e gerar o relatório final."""
    try:
        model_name = "gemini-1.5-flash-latest"
    except Exception as e:
        print(f"Não foi possível listar modelos para o Revisor: {e}. Usando modelo padrão.")
        model_name = "gemini-pro" # Fallback

    revisor = Agent(
        name="Agente_Revisor",
        model=model_name, # Usa o modelo definido
         instruction=f"""
        Você é o REVISOR, o responsável por consolidar todas as informações processadas pelos agentes anteriores e gerar o RELATÓRIO FINAL formal da ideia Kaizen.
        Sua função é:
        1. Coletar e organizar as informações do 'AUXILIADOR' e do 'TÉCNICO', juntamente com os dados básicos da ideia (ID, Data, Funcionário).
        2. Definir o 'STATUS ATUAL DA IDEIA' inicial como "Pronto para Avaliação Técnica".
        3. Formatar todas essas informações em um relatório claro, organizado e fácil de ler, ideal para ser "printado" (capturado a tela).
        4. Incluir o logotipo textual "SADA" no topo do relatório.

        O RELATÓRIO FINAL deve seguir EXATAMENTE este formato. Mantenha a estrutura, quebras de linha e marcadores.

        --- RELATÓRIO DA IDEIA KAİZEN ---
        * S A D A *
        * (Sistema de Análise e *
        * Desenvolvimento de Aprimoramentos) *
        -----------------------------------
        ID da Ideia: {idea_id}
        Data do Registro: {data_registro}
        -----------------------------------
        FUNCIONÁRIO:
        Nome: {nome}
        Matrícula: {matricula}
        Setor (Ideia): {setor}
        -----------------------------------
        DETALHES DA IDEIA:
        [Copie e formate aqui o conteúdo da seção 'IDEIA APRIMORADA' do Auxiliador]
        -----------------------------------
        ANÁLISE INICIAL DOS AGENTES:
        [Copie e formate aqui o conteúdo da seção 'ANÁLISE DE SIMILARIDADES' do Auxiliador]

        [Copie e formate aqui o conteúdo da seção 'ANÁLISE TÉCNICA PRELIMINAR' do Técnico]
        -----------------------------------
        STATUS ATUAL DA IDEIA:
        Status: Pronto para Avaliação Técnica
        -----------------------------------
        --- FIM DO RELATÓRIO ---

        Aqui estão as informações dos agentes anteriores que você deve usar para preencher o relatório:

        Informações do AUXILIADOR:
        {auxiliador_output}

        Informações do TÉCNICO:
        {tecnico_output}

        Sua resposta deve ser APENAS o texto completo do RELATÓRIO FINAL formatado como especificado acima. Não adicione nenhum texto antes ou depois do relatório.
        """,
        description="Agente que consolida as análises e gera o relatório final da ideia Kaizen."
    )
    print("Revisor processando...")
    return call_agent(revisor) # Passa mensagem vazia.


# --- Fluxo Principal do Programa ---
if __name__ == "__main__":
    print("🚀 Iniciando o Sistema SADA de Gestão de Ideias Kaizen 🚀")

    while True:
        # 1. Coleta inicial de dados do funcionário e da ideia bruta
        nome_funcionario, matricula_funcionario, setor_da_ideia = iniciar_chat()
        problema_identificado, solucao_proposta = coletar_ideia()

        # --- Processamento da Ideia através da Sequência de Agentes ---
        print("\n--- Iniciando Processamento da Ideia com Agentes ---")

        # Obtém o resumo das ideias existentes para o Auxiliador
        existing_summary = get_existing_ideas_summary()

        # 2. Chama o Agente AUXILIADOR
        auxiliador_result_text = call_auxiliador_agent(
            nome=nome_funcionario,
            matricula=matricula_funcionario,
            setor=setor_da_ideia,
            problema_bruto=problema_identificado,
            solucao_bruta=solucao_proposta,
            existing_ideas_summary=existing_summary
        )
        # Verifica se o Auxiliador retornou erro
        if "[ERRO NO AGENTE Agente_Auxiliador:" in auxiliador_result_text:
             print("❌ Falha no processamento pelo Auxiliador. Abortando processamento.")
             continue # Volta para o início do loop para nova ideia ou sair

        print("✅ Auxiliador concluído.")
        # Opcional: display(Markdown(f"### Saída Auxiliador:\n{auxiliador_result_text}")) # Debug


        # 3. Chama o Agente TÉCNICO, passando o resultado do Auxiliador
        tecnico_result_text = call_tecnico_agent(
            auxiliador_output=auxiliador_result_text,
            setor_da_ideia=setor_da_ideia # Passamos o setor também, pois é relevante para o direcionamento
        )
        # Verifica se o Técnico retornou erro
        if "[ERRO NO AGENTE Agente_Tecnico:" in tecnico_result_text:
             print("❌ Falha no processamento pelo Técnico. Abortando processamento.")
             continue # Volta para o início do loop

        print("✅ Técnico concluído.")
        # Opcional: display(Markdown(f"### Saída Técnico:\n{tecnico_result_text}")) # Debug

        # 4. Prepara dados para o Agente REVISOR (ID, Data/Hora)
        global idea_counter # Usa a variável global
        idea_counter += 1 # Incrementa o contador para o novo ID
        current_idea_id = idea_counter
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # 5. Chama o Agente REVISOR, passando os resultados dos agentes anteriores e os dados finais
        revisor_final_report_text = call_revisor_agent(
            idea_id=current_idea_id,
            data_registro=timestamp,
            nome=nome_funcionario,
            matricula=matricula_funcionario,
            setor=setor_da_ideia,
            auxiliador_output=auxiliador_result_text,
            tecnico_output=tecnico_result_text
        )
        # Verifica se o Revisor retornou erro
        if "[ERRO NO AGENTE Agente_Revisor:" in revisor_final_report_text:
             print("❌ Falha ao gerar o relatório pelo Revisor. Abortando processamento.")
             continue # Volta para o início do loop

        print("✅ Revisor concluído. Relatório gerado.")
        print("\n--- Fim do Processamento com Agentes ---\n")

        # --- Armazenamento e Exibição ---

        # 6. Armazena a ideia processada no banco de dados em memória
        # Armazenamos o relatório final gerado para fácil recuperação,
        # além dos dados brutos e as saídas dos agentes intermediários, se necessário.
        banco_de_ideias.append({
            "id": current_idea_id,
            "data_registro": timestamp,
            "nome_funcionario": nome_funcionario,
            "matricula": matricula_funcionario,
            "setor": setor_da_ideia,
            "problema_bruto": problema_identificado,
            "solucao_bruta": solucao_proposta,
            "auxiliador_output": auxiliador_result_text, # Opcional: guardar outputs intermediários
            "tecnico_output": tecnico_result_text,     # Opcional: guardar outputs intermediários
            "revisor_report_text": revisor_final_report_text, # O relatório final em texto
            "status": "Pronto para Avaliação Técnica" # Status definido pelo fluxo
        })

        # 7. Exibe o relatório final formatado e CENTRALIZADO
        print("\n--- RELATÓRIO FINAL GERADO ---")
        # Cria um bloco HTML para centralizar o conteúdo
        # Usamos <pre> para manter a formatação do texto do relatório (quebras de linha, espaços)
        # e alinhamos o texto da <pre> à esquerda dentro do bloco centralizado.
        centered_html_report = f"""
        <div style="text-align: center;">
            <pre style="text-align: left; display: inline-block; border: 1px solid #ccc; padding: 10px; background-color: #f9f9f9; white-space: pre-wrap; word-wrap: break-word;">{revisor_final_report_text}</pre>
        </div>
        """
        display(HTML(centered_html_report))
        print("\nVocê pode tirar um print desta tela para seus registros.")
        print("-------------------------------\n")


        # 8. Pergunta se o usuário deseja registrar outra ideia (mantendo input s/n)
        # Nota: Uma verdadeira "múltipla escolha sem digitar" exigiria bibliotecas de UI mais complexas (ex: ipywidgets),
        # que aumentariam significativamente o código. Mantemos o input s/n como a opção mais simples aqui.
        while True:
            continuar = input("Deseja registrar outra ideia? Digite 's' para SIM ou 'n' para NÃO: ").lower().strip()
            if continuar in ['s', 'n']:
                break
            else:
                print("Opção inválida. Por favor, digite apenas 's' ou 'n'.")

        if continuar == 'n':
            print("\nObrigado por usar o SADA! Suas ideias são muito importantes.")
            # Opcional: Mostrar todas as ideias registradas no final (pode ser útil para debug)
            # print("\n--- Todas as Ideias Registradas (Resumo) ---")
            # if banco_de_ideias:
            #     for ideia in banco_de_ideias:
            #         print(f"ID: {ideia['id']} - {ideia['data_registro']} - {ideia['nome_funcionario']} - Status: {ideia['status']}")
            # else:
            #     print("Nenhuma ideia foi registrada nesta sessão.")
            # print("---------------------------------------------")
            break # Sai do loop principal

        print("\n" + "="*70 + "\n") # Separador para a próxima ideia